In [18]:
import tdmclient.notebook
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning
import obstacle_avoidance

In [19]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def get_prox():
    global prox_horizontal
    obst = prox_horizontal
    return obst

In [28]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""


TRESH_DIST = 20 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436 #150 speed
YAW_TO_MMS = 0.06086 # -100 100
SPEED_OFFSET = 5

def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        # we need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        print("Px_to_mm= ", px_to_mm)
        
        px_to_mm = 0.64
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_obstacle.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_path.jpg', frame_init) #POUR TUNING PAR ELIOTT

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame_width = int(cap.get(3))

        frame_height = int(cap.get(4))

        # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.

        out = cv2.VideoWriter('project_video.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1920,1080))
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame.
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        # x_est_t_minus_1 = np.array([0.0,0.0,0.0])
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[0.1,  0,   0],
                                [  0,0.1,   0],
                                [  0,  0, 0.1]])
    
    
        #update le dt car sinon kalman marche pas
        dt = 0.1 # EN SECONDES POUR KALMAN!!!!! same as PI_CLOCK of pid defined in filters?
        sum_error = 0 
        alt_error_pid = 0
        v_l = 0
        v_r = 0
        
        state = 0
        obstThrH = 1400
        
        time.sleep(0.1)
        
        while len(goal_list)!=0:
            
            #print("goal is: ", goal_list[0])
            
            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, is_from_camera = True)
            
            
#             print(f'Timestep measurement={obs_vector_z_t}')
    
            # Run the Extended Kalman Filter and store the 
            # near-optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.ekf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Time interval
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, is_from_camera = False)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, is_from_camera = 2)
            
            out.write(frame)
            frame = cv2.resize(frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
                
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            #check if obstacle in coming
            obst=get_prox()
            
            if state == 0:
                for i in range(len(obst)-2):
            # switch from goal tracking to obst avoidance if obstacle detected
                    if (obst[i] > obstThrH):
                        print('obst', obst)
                        state = 1
            if state==0:
                sum_error, alt_error_pid, v_l,v_r = filters.pid(optimal_state_estimate_t,goal_list[0],sum_error, alt_error_pid,dt)
    
            else:  #if yes, obstacle avoidance
                v, state = obstacle_avoidance.obstacle_avoidance(obst, v_l, v_r, True)
                v_l = v[0]
                v_r = v[1]
            
            v = (v_l + v_r)*SPEED_TO_MMS/2*1.1
            yaw = (v_l-v_r)*YAW_TO_MMS*4
            u_t_minus_1 = [v, yaw]; # delta_v returned by PID
                
            set_speed(int(v_l), int(v_r+SPEED_OFFSET))

            actual = time.time()
            diff = actual-previous
#             print("elapsed time is: ", diff)
            if diff<dt:
                time.sleep(dt-diff)
            
    finally: 
        out.release()
        set_speed(0, 0)
        cv2.destroyAllWindows()



In [54]:
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [57]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [58]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [29]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)
importlib.reload(obstacle_avoidance)

<module 'obstacle_avoidance' from 'C:\\Users\\Juliette\\Documents\\_MobileRobotics\\Mobile_robotics\\obstacle_avoidance.py'>

In [32]:
main()

take the rigth image
Px_to_mm=  0.4731223429726782


100%|██████████| 1/1 [00:00<00:00, 77.56it/s]


angle du robot 47.81555668421123
angle du robot 48.22041650262898
angle du robot 49.361963664640456
angle du robot 49.11264935996941
angle du robot 52.86557466344257
angle du robot 55.80263046971582
angle du robot 59.541873785327496
angle du robot 64.42475225928337
angle du robot 68.51375339500964
angle du robot 71.27502858053849
angle du robot 75.41633870486636
angle du robot 77.59169483176325
angle du robot 78.91376754317471
angle du robot 80.86152534951482
angle du robot 84.53184591776717
angle du robot 87.13038155158519
angle du robot 86.06738221744662
angle du robot 86.24203522162313
angle du robot 86.71955009757914
angle du robot 86.75454453606686
angle du robot 87.37756403262637
angle du robot 88.5775071919923
angle du robot 88.06404684144741
angle du robot 89.2361541442112
angle du robot 87.03769249259643
angle du robot 87.44377546473994
angle du robot 87.42477065307659
angle du robot 86.76370676377692
angle du robot 88.51489514222341
angle du robot 87.46399925500887
angle du r

angle du robot 257.17838438451383
angle du robot 257.3084113714363
angle du robot 256.48608184121787
angle du robot 255.8540362618149
angle du robot 255.76575918471298
angle du robot 255.68145780181965
angle du robot 255.7846001168865
angle du robot 255.07990517408018
angle du robot 255.29350504384252
angle du robot 256.623963001703
angle du robot 256.99009270013147
angle du robot 255.17182456506384
angle du robot 254.29146113439432
angle du robot 253.30202195261452
angle du robot 254.2889636748787
angle du robot 253.4537074992666
angle du robot 253.31592140712868
angle du robot 251.23110340407337
angle du robot 247.19626138120285
angle du robot 245.34207088459985
angle du robot 242.04408261085902
angle du robot 238.0581522149898
angle du robot 236.93306275288674
angle du robot 231.64017973100846
angle du robot 229.51838788660237
angle du robot 226.50560879206688
angle du robot 225.44968765445168
angle du robot 224.49138706619468
angle du robot 221.10916600623352
angle du robot 218.816

IndexError: list index out of range

In [33]:
cap.release()

In [34]:
await tdmclient.notebook.stop()